#### 批量修改文件夹的名字, 去掉汉字, 都带上_frms
1. 支持多级目录修改为只有单一目录
2. 去掉空格
3. 去掉汉字
4. 增加_frms


In [71]:
import os
import string
import shutil
import pathlib

In [7]:

def add_prefix_to_name(ori_name="", prefix="_frms", judge_before_do=True):
    if not ori_name:
        print("no name specified")
        return 
    #ignore space tab
    char_sets = set(string.printable)
    set.remove(char_sets, ' ')
    set.remove(char_sets, '\t')
    striped = filter(lambda x: x in char_sets, ori_name)
    ori_name = ''.join(list(striped))
    if judge_before_do:
        new_name = ori_name + prefix if prefix not in ori_name else ori_name
    else:
        new_name = ori_name + prefix
        
    return new_name

#add_prefix_to_name(r)

def change_all_folder_name(root_dir="", rename_func=None):
    if not root_dir or not os.path.isdir(root_dir):
        print("Invaild Root Path")
        return
    if not rename_func:
        print("rename_function must specified")
        return 
    items = os.listdir(root_dir)
    sub_dirs = [i for i in items if os.path.isdir(i)]
    #print(sub_dirs)
    new_dirs = [rename_func(i) for i in sub_dirs]
    #os.rename(sub_dirs[0], new_dirs[0])
    for x,y in zip(sub_dirs, new_dirs):
        print(f"rename {x} to {y}")
        os.rename(x, y)
    return 
    


In [75]:
#多级目录改为一级目录
def no_folder_in_dir(dir_name=""):
    """
    判断是否为最底层文件夹
    """
    if not dir_name or not os.path.isdir(dir_name):
        print(f"Invaild Path:{dir_name}")
        return False
    items = os.listdir(dir_name)
    sub_dirs = [i for i in items if os.path.isdir(os.path.join(dir_name,i))]
    
    if len(sub_dirs) != 0:
        #print(f"\tFalse...sub_dirs:{sub_dirs} ")
        return False
    else:
        #print(f"\tTrue...sub_dirs:{sub_dirs} ")
        return True

def copy_all_files_then_del_dir(dir_name=""):
    """
    因为当dir0/dir1中dir0==dir1的时候,是没办法直接用dir1覆盖dir0的,所以需要操作dir1中所有文件到dir0,然后删除dir1
    """
    if not dir_name or not os.path.isdir(dir_name):
        print(f"Invaild Path:{dir_name}")
        return False
    items = os.listdir(dir_name)
    sub_files = [i for i in items if os.path.isfile(os.path.join(dir_name,i))]
    print(f"total: {len(sub_files)} files")
    for f in sub_files:
        src_file = os.path.join(dir_name, f)
        dst_file = os.path.join(os.path.dirname(dir_name), f)
        shutil.copyfile(src_file, dst_file)
    shutil.rmtree(dir_name)
    print(f"删除路径名字: {dir_name}")
    return True
    
def multidir2singledir(root_dir="", work_dir="", judge_condition=None):
    """
        将该路径下所有文件夹嵌套的都删掉,改为单一一层
    """
    print(f"{root_dir}\n{work_dir}\n{'*'*20}")
    if not root_dir or not os.path.isdir(root_dir):
        print(f"Invaild Root Path:{root_dir}")
        return
    if not work_dir or not os.path.isdir(work_dir):
        print(f"Invaild Work Path:{work_dir}")
        return
    if judge_condition is None:
        print("judge condition must here!")
        return
    _level = 0
    if root_dir == work_dir:
        _level = 0
        print(f"the first level...")
    else:
        _levels = set(work_dir.split(os.path.sep))- set(root_dir.split(os.path.sep))
        #print(f"level{_level}...{set(root_dir.split(os.path.sep))}=={set(work_dir.split(os.path.sep))}")
        _level = len(_levels)  
        print(f"levels{_level}...{_levels}")
    items = os.listdir(work_dir)
    sub_dirs = [i for i in items if os.path.isdir(os.path.join(work_dir, i))]
          
    for subd in sub_dirs:
        if judge_condition((os.path.join(work_dir,subd))):
            if 0 == _level:
                print(f"{'#'*(_level+1)}do not need process...{subd}")
                continue
            else:
                #copy file 2 upper-level
                normal_name = add_prefix_to_name(subd, "_frms", True)
                src_dir = os.path.join(work_dir, subd)
                dst_dir = os.path.join(os.path.split(work_dir)[0], normal_name)
                ret = copy_all_files_then_del_dir(src_dir)
                      
                #path = pathlib.Path(src_dir)
                #path.replace(dst_dir)
                #shutil.move(src_dir, dst_dir)
                return _level
        else:
            multidir2singledir(root_dir, os.path.join(work_dir, subd), judge_condition)
    return None
        
    #1. 到最后一层复制所有文件到当前层
    #2. 删除当前层包含的文件夹

In [78]:
#运行修改文件夹的名字---
r = r'/mnt/g/LabelData/Plaque/PlaqueV9.0'
change_all_folder_name(r, add_prefix_to_name)


In [83]:
#保留单层路径
r = r'/mnt/g/LabelData/Plaque/PlaqueV9.0'
multidir2singledir(r, r, no_folder_in_dir)

/mnt/g/LabelData/Plaque/PlaqueV9.0
/mnt/g/LabelData/Plaque/PlaqueV9.0
********************
the first level...
#do not need process...06_Plaque_XX_YY_18_frms
#do not need process...06_Plaque_XX_YY_19_frms
#do not need process...06_Plaque_XX_YY_20_frms
#do not need process...06_Plaque_XX_YY_21_frms
#do not need process...06_Plaque_XX_YY_22_frms
#do not need process...06_Plaque_XX_YY_23_frms
#do not need process...06_Plaque_XX_YY_24_frms
#do not need process...06_Plaque_XX_YY_25_frms
#do not need process...06_Plaque_XX_YY_26_frms
#do not need process...06_Plaque_XX_YY_27_frms
#do not need process...06_Plaque_XX_YY_28_frms
#do not need process...06_Plaque_XX_YY_29_frms
#do not need process...06_Plaque_XX_YY_30_frms
#do not need process...06_Plaque_XX_YY_31_frms
#do not need process...06_Plaque_XX_YY_32_frms
#do not need process...06_Plaque_XX_YY_33_frms
#do not need process...06_Plaque_XX_YY_34_frms
#do not need process...06_Plaque_XX_YY_35_frms
#do not need process...06_Plaque_XX_YY_36_fr

In [82]:
!python --version

Python 3.6.8


In [ ]:

s = "asdfasj发哈是"
fil = filter(lambda x: x in string.printable, s)
subs = ''.join(list(fil))